##  Whole-body locomotion 

The objective of this exercise is to demonstrate the capabilities of Crocoddyl to generate gaited locomotion patterns. We use for that some wrapping scripts defined in Crocoddyl. They have later been extended into the [https://gitlab.laas.fr/memory-of-motion/sobec](Sobec package), which you can also consider.

## Set up

The tuto is based on Crocoddyl, we don't need anything new.

In [ ]:
import example_robot_data
import numpy as np
import pinocchio
import crocoddyl
from crocoddyl.utils.biped import SimpleBipedGaitProblem

## Problem definition
The optimal control problem enforces the gait, i.e. the contact phases for each time step are defined beforehand and cannot be changed by the solver.
The dynamics is the solution to the "Gauss principled" QP:

$$
\left[\begin{matrix}
 \mathbf{M} & \mathbf{J}^{\top}_c \\
 {\mathbf{J}_{c}} & \mathbf{0} \\
\end{matrix}\right]
\left[\begin{matrix}
 \dot{\mathbf{v}} \\ -\boldsymbol{\lambda}
\end{matrix}\right]
 = 
\left[\begin{matrix}
  \boldsymbol{\tau} - \mathbf{h} \\
  -\dot{\mathbf{J}}_c \mathbf{v} \\
\end{matrix}\right]$$.

Base on a predefined walking gait, we build per each phase a specific multi-contact dynamics. Indeed we need to describe multi-phase optimal control problem. One can formulate the multi-contact optimal control problem (MCOP) as follows:


$$\mathbf{X}^*,\mathbf{U}^*=
\begin{Bmatrix} \mathbf{x}^*_0,\cdots,\mathbf{x}^*_N \\
				  \mathbf{u}^*_0,\cdots,\mathbf{u}^*_N
\end{Bmatrix} =
\arg\min_{\mathbf{X},\mathbf{U}} \sum_{p=0}^P \sum_{k=1}^{N(p)} l_p(\mathbf{x},\mathbf{u})$$
subject to
$$ \mathbf{{x_{t+1}}} = \mathbf{f}_t(\mathbf{x},\mathbf{u}), \text{for } t \in [1..N]$$

where $\mathbf{f}$ is the solution to the Gauss QP above.

`SimpleBipedGaitProblem` class builds action models for each locomotion phase:
  - `createSwingFootModel`: defines an action model for the swing phase
  - `createFootSwitchModel`: defines an action model for switch knots between phases
  
Then we build a walking by combining a set of contact phases and their contact switches. This is defined by `createFootstepModel`

Now let's create a walking OC problem for the Talos legs.

## Implementation

The problem is defined from a set of parameter describing the expected walk.

In [ ]:
# Creating the lower-body part of Talos
robot = example_robot_data.load("talos_legs")

# Setting up the 3d walking problem
rightFoot = "right_sole_link"
leftFoot = "left_sole_link"

# Create the initial state
q0 = robot.q0.copy()
v0 = pinocchio.utils.zero(robot.model.nv)
x0 = np.concatenate([q0, v0])


# Creating the walking problem
stepLength = 0.6  # meters
stepHeight = 0.1  # meters
timeStep = 0.0375  # seconds
stepKnots = 20
supportKnots = 10

We gathered the details of the implementation of the OCP in the SimpleBipedGaitProblem class, located in the [repository of Crocoddyl](https://github.com/loco-3d/crocoddyl/blob/devel/bindings/python/crocoddyl/utils/biped.py).

In [ ]:
gait = SimpleBipedGaitProblem(robot.model, rightFoot, leftFoot)
problem = gait.createWalkingProblem(
    x0, stepLength, stepHeight, timeStep, stepKnots, supportKnots
)

You can take a look at the resulting OCP by scrolling inside the nested models. Alternativelly, you can use the convenient "repr" functions to display the OCP with all details as a string.

In [ ]:
from tp10.repr_ocp import printReprProblem
np.set_printoptions(precision=3, linewidth=350, suppress=True,threshold=1e4)


In [ ]:
printReprProblem(problem)

## Solve and display

The solve of the problem is classical, we use the FDDP solver.

In [ ]:
# Solving the 3d walking problem using Feasibility-prone DDP
ddp = crocoddyl.SolverFDDP(problem)

# Using the meshcat displayer, you could enable gepetto viewer for nicer view
# display = crocoddyl.GepettoDisplay(robot, 4, 4)
ddp.setCallbacks(
    [
        crocoddyl.CallbackLogger(),
        crocoddyl.CallbackVerbose(),
    ]
)

In [ ]:
# Solve the optimal control problem
ddp.th_stop = 1e-9
init_xs = [x0] * (problem.T + 1)
init_us = []
ddp.solve(init_xs, init_us, maxiter=100, is_feasible=False)

The convergence of the solver should be quite straight-forward, feew steps, mostly of length 1, converging to nearly 0 gradient.

In [ ]:
# Plotting the solution and the DDP convergence
log = ddp.getCallbacks()[0]
crocoddyl.plotConvergence(
    log.costs, log.pregs, log.dregs, log.grads, log.stops, log.steps
)

Finally we can visualize the solution.

In [ ]:
from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.viewer.jupyter_cell()

In [ ]:
viz.play([x[:robot.model.nq] for x in ddp.xs],timeStep)